In [1]:
import nbimporter
import pandas as pd
import numpy as np
import awswrangler as wr
import math
import os
import boto3
from os import listdir
from os.path import isfile, join

from cliofunctions import ac_aws, normalize, normalize2, sort_and_group, operation, extract_info, porcentaje, promedio, ranking, top

# JUNAEB
El siguiente script extrae los datos del mapa nutricional por comuna durante el año 2020 declarados por la JUNAEB. Notar que al imprimir el archivo, la columna que tiene los nombres de las comunas tiene el nombre de "COMUNA", es importante definirlo como "main_groupe".

In [2]:
excel_file = pd.ExcelFile("MAPA-NUTRICIONAL-2020-POR-RBD.xlsx")
sheet_names = excel_file.sheet_names[:-1]

In [3]:
main_df =  pd.read_excel("MAPA-NUTRICIONAL-2020-POR-RBD.xlsx", sheet_name = sheet_names[0], header=None)
main_groupe = "COMUNA"

## Arreglo de datos
En las siguientes lineas se realiza la preparación de los datos para que estos puedan estar listos para ser agregados a la matriz consulta

In [4]:
main_df.iloc[0,:].dropna(axis = 0)
row_1 = main_df.iloc[0,:].to_list()
row_1 = ["Cambiar" if x != x else x for x in row_1]
rep = "ELIMINAR"

for i in range(len(row_1)):
    if row_1[i] == "Cambiar":
        row_1[i] = rep
    else:
        rep = row_1[i]

for i in range(len(row_1)):
    if "Prevalencia niñas" in row_1[i]:
        row_1[i] = "% Niñas"
    if "Prevalencia niños" in row_1[i]:
        row_1[i] = "% Niños"
    if "Prevalencia total" in row_1[i]:
        row_1[i] = "% Total Estudiantes"
    if "niños" in row_1[i]:
        row_1[i] = "Niños"
    if "niñas" in row_1[i]:
        row_1[i] = "Niñas"
    if "estudiantes" in row_1[i]:
        row_1[i] = "Estudiantes"

In [5]:
header = []
keep_header = []
aux = main_df.iloc[1,:].dropna(axis = 0)
for i in range(len(row_1)):
    if i == 5:
        header.append("COMUNA")
    else:
        header.append(row_1[i] + " " + aux[i])

for encabezado in header:
    if "ELIMINAR" not in encabezado:
        keep_header.append(encabezado)


In [6]:
aux_df = main_df.iloc[2:,:]
aux_df.columns = header
main_df = aux_df[keep_header]
replace_head = ["TOTAL ESTUDIANTES", "TOTAL NIÑOS", "TOTAL NIÑAS"]
for j in range(len(keep_header)):
    for excepcion in replace_head:
        if excepcion in keep_header[j]:
            keep_header[j] = excepcion
main_df.columns = keep_header
main_df = main_df.iloc[:-1,:]

In [7]:
df = main_df.groupby("COMUNA").sum()

In [8]:
cols = df.columns.to_list()
for i in range (3):
    a = i * 17
    b = cols[a : a + 17]
    for j in range(1,9):
        df[b[j+8]] = df[b[j-1]] / df[b[8]]
main_df = df

### Descriptor

In [9]:
rows, cols = main_df.shape
total_indicadores = cols-1
header_d = ["DIMENSIÓN", "SUBDIMENSIÓN", "VARIABLE", "INDICADOR", "DESCRIPCIÓN", "NOMBRE VARIABLE", "FUENTE", "AÑO", "ALCANCE"]
DIM = "SOCIAL"
SUBD = "SALUD"
VAR = "COMORBILIDADES"
FUENTE = "JUNAEB"
AÑO = "2020"
ALCANCE = "COMUNAL"
n_var = main_df.columns
condicion = aux[10:].to_list()
descripcion = []
for k in range(len(n_var)):
    a = "LA CANTIDAD"
    c = " CON "
    if "%" in n_var[k]:
        a = "EL PORCENTAJE"
    if "Niñas" in n_var[k]:
        b = " DE SEXO FEMENINO"
    if "Niños" in n_var[k]:
        b = " DE SEXO MASCULINO"
    if "Estudiantes" in n_var[k]:
        b = " "
    if "Normal" in n_var[k]:
        c = " CON CONDICIÓN "
    d = condicion[k].upper()
    for excepcion in replace_head:
        if excepcion in d:
            c = ""
            d = ""
    descripcion.append("INDICA " + a + " ESTUDIANTES" + b + c + d)

In [10]:
descriptor = pd.DataFrame(columns=header_d)
for k in range(len(n_var)):
    list_desc = DIM, SUBD, VAR, n_var[k].upper(), descripcion[k], condicion[k].upper() ,FUENTE, AÑO, ALCANCE
    descriptor.loc[k] = list_desc

### Archivos complementarios
CSV que indican orden de comunas, de región o excepciones para reemplazar en los nombres de las comunas y/o regiones de cada dataset. Notar que en este caso el archivo tiene un alcance COMUNAL.

In [11]:
sort_by_comuna = "s3://test1-rvc/Sort_by_comuna.csv"
sort_by_region = "s3://test1-rvc/Sort_by_region.csv"
replace_sort = "s3://test1-rvc/Sort_replace_weird_objects.csv"

#reading dataframe de orgen (regional o comunal) y matriz de reemplazos (para singularidades en comunas o regiones)
reach = "COMUNAL"
if reach == "COMUNAL":
    sort_df = pd.read_csv(sort_by_comuna, sep = ";")
elif reach == "REGIONAL":
    sort_df = pd.read_csv(sort_by_region, sep = ";")
else:
    print("Debe indicar alcance: REGIONAL O COMUNAL")
replace = pd.read_csv(replace_sort, sep = ";")

### Juntar el descriptor con los datos

In [12]:
main_df = sort_and_group(main_df, main_groupe, sort_df, replace).reset_index()
df_add = pd.concat((descriptor,main_df),axis=1)
del df_add["index"]

In [13]:
MC = "s3://test1-rvc/Matriz_consulta.csv"
matrix_df = pd.read_csv(MC, sep = ",")

In [14]:
df_add.columns = matrix_df.columns
matrix_df_act = pd.concat((matrix_df,df_add),axis=0)

# Exportar
Se exporta la Matriz de consulta con los datos de CONAF agregados

In [15]:
matrix_df_act.to_csv('Matriz_consulta.csv', index=False, encoding='utf-8-sig')